In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import spacy
import gensim
import pyLDAvis.gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

# Folder
folder = './training/Final/'

file =  folder + 'dataset.csv'

%matplotlib inline

In [2]:
frame = pd.read_csv(file, sep='|')
frame.head()

,content,industries,location,title,type_empl,url,wage,suburb,experiences,title_left_over
0,Public Safety Business Agency / Kedron Tempora...,Information & Communication Technology,Brisbane,DA,Permanent,https://www.seek.com.au/job/39018343?type=stan...,85572.0,Northern Suburbs,General,Data Analyst
1,Fantastic working culture Flexible work enviro...,Information & Communication Technology,Brisbane,SDA,Contract,https://www.seek.com.au/job/39007973?type=stan...,99961.0,CBD & Inner Suburbs,Senior,Senior Business Analyst
2,CBD Location 12 month contract Join this State...,Information & Communication Technology,Brisbane,DA,Contract,https://www.seek.com.au/job/38912693?type=stan...,174590.0,CBD & Inner Suburbs,General,Database Analyst
3,CBD Location Initial 3 month contract Join thi...,Information & Communication Technology,Brisbane,DA,Contract,https://www.seek.com.au/job/39037731?type=stan...,177450.0,CBD & Inner Suburbs,General,Data Analyst
4,CBD Location! Initial contract through till en...,Information & Communication Technology,Brisbane,SDA,Contract,https://www.seek.com.au/job/39003265?type=stan...,173550.0,CBD & Inner Suburbs,Senior,Senior Test Analyst


In [3]:
process_data = frame['content'].values
len(process_data)

1569

In [4]:
m_STOPWORD = gensim.parsing.preprocessing.STOPWORDS
extra_STOPWORD = ['digit','agil','detail']
m_STOPWORD = m_STOPWORD.union(extra_STOPWORD)

In [5]:
def lemmatize_stemming(text):
    stemmer = SnowballStemmer("english")
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text):
        # Give word . Return the fequency of word\n",
        # Exclude STOPWORD, everything very less then 3\n",
        if token not in m_STOPWORD and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

In [6]:
processed_docs = []
for doc in process_data:
    processed_docs.append(preprocess(doc))

In [7]:
# Push to dictionary
dictionary = gensim.corpora.Dictionary(processed_docs)
# "# I dont want words no_below, no_above
dictionary.filter_extremes(no_below=10, no_above=0.2,keep_n=2000)

In [8]:
# "# Bag of words
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [9]:
# Fit LDA model\n",
# num_topic [dont know] sothat pick a proper number,
lda_model = gensim.models.LdaMulticore(bow_corpus,
                                       num_topics=6,
                                      id2word=dictionary,
                                     passes=10, workers=2)

In [10]:
# LDA help vizualize
lda_display = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
pyLDAvis.display(lda_display)

C:\Users\Shaynie\Anaconda3\envs\capstone_app\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
